In [ ]:
import psycopg2

In [ ]:
import psycopg2.extensions

In [ ]:
dsn = "postgresql://michael:@/landuse?host=/run/postgresql"

In [ ]:
conn = psycopg2.connect(**psycopg2.extensions.parse_dsn(dsn))

In [ ]:
def send_sql(connection, query, **kwargs):
    import time
    args = kwargs.get("args")
    quiet = kwargs.get("quiet", False)
    return_result = kwargs.get("return_result", False)
    start = time.time()
    cursor = connection.cursor()
    result = None
    rowcount = -1
    try:
        cursor.execute(query, args)
        rowcount = cursor.rowcount
        if return_result:
            result = cursor.fetchall()
        connection.commit()
    except Exception as ex:
        print("Failed to execute SQL statement:\n{}\nReason: {}".format(cursor.query, ex))
        connection.rollback()
    finally:
        cursor.close()
        msg = "Executed SQL command in {:0.3f} seconds.".format(time.time() - start)
        if rowcount >= 0:
            msg = "{}, affected {} rows.".format(msg[:-1], rowcount)
        print(msg)
    return result

## Prepare database

We have to make the area_id column of the table `land` unique and calculate the area of each polygon.

In [ ]:
send_sql(conn, "ALTER TABLE land ADD CONSTRAINT area_id_unique UNIQUE(area_id);")

## Municipalities as polygons

First, we have to get the polygon of Germany

In [ ]:
sql = "SELECT geom FROM admin_boundaries WHERE admin_level = 4 AND name = 'Baden-Württemberg';"
germany_wkb = send_sql(conn, sql, return_result=True)[0]
#c.execute(sql)
#germany_wkb = c.fetchone()[0]

Create a database table of municipalities

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS cities;")

In [ ]:
send_sql(conn, """SELECT
    area_id, admin_level, name, tags, geom
  INTO cities
  FROM admin_boundaries
  WHERE admin_level = 8 AND ST_Intersects(geom, %s);
  CREATE INDEX ON cities USING gist(geom);
""", args=(germany_wkb,))
send_sql(conn, "ALTER TABLE cities ADD CONSTRAINT cities_area_id_unique UNIQUE(area_id);")

Add "kreisfreie Städte" (AL 6 polygons with no AL 8 inside them):

In [ ]:
sql = """INSERT INTO cities (area_id, admin_level, name, tags, geom)
  SELECT
      area_id,
      admin_level,
      name,
      tags,
      geom
    FROM admin_boundaries AS a
    WHERE
      admin_level = %s
      AND (SELECT c.geom FROM cities AS c WHERE ST_Contains(a.geom, c.geom) LIMIT 1) IS NULL;"""
for level in [6, 4]:
    send_sql(conn, sql, args=(level,))

Create index on name and area_id for later use.

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS cities_area_id_name_idx ON cities USING btree(area_id, name);")

## Overlaps of landuse

Find intersections between landuse polygons:

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS land_area_size_idx ON land USING btree(area);")
send_sql(conn, "DROP TABLE IF EXISTS landuse_overlaps;")
sql = """CREATE TABLE landuse_overlaps AS
SELECT
    ST_CollectionExtract(ST_Intersection(a.geom, b.geom), 3) AS geom,
    a.area_id AS id1,
    b.area_id AS id2,
    COALESCE(a.landuse, a."natural", a.waterway, a.water, a.leisure, a.military, a.amenity, a.man_made) AS feature1,
    COALESCE(b.landuse, b."natural", b.waterway, b.water, b.leisure, b.military, b.amenity, b.man_made) AS feature2,
    a.area AS size_1,
    b.area AS size_2
  FROM land AS a
  JOIN land AS b
    ON
    a.area >= b.area
    AND a.geom && b.geom
    AND ST_Intersects(a.geom, b.geom)
    AND ST_Relate(a.geom, b.geom, 'T********')
    AND a.area_id != b.area_id;
"""
send_sql(conn, sql)

Create a matrix of feature classes overlapping each other by area:

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_overlaps_feature_classes;")
sql = """CREATE TABLE landuse_overlaps_feature_classes AS
SELECT
    feature1,
    feature2,
    SUM(ST_Area(geom::geography)) AS area_size,
    COUNT(1) AS occurences
  FROM landuse_overlaps
  GROUP BY feature1, feature2;
"""
send_sql(conn, sql)

## Generic stats about landuses (sizes, node count, boundary length)

Get boundary lines of landuse polygons (as (multi)linestrings) and the area size.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS landuse_boundary_lines;")
sql = """CREATE TABLE landuse_boundary_lines AS
SELECT
    ST_Boundary(geom) AS geom,
    ST_Length(ST_Boundary(geom)::geography) AS length,
    ST_NPoints(geom) AS node_count,
    ST_Area(geom::geography) AS area_size,
    area_id,
    COALESCE(landuse, "natural", waterway, water) AS feature,
    tags
  FROM land;
"""
send_sql(conn, sql)
#send_sql(conn, "CREATE INDEX landuse_boundary_lines_length_idx ON landuse_boundary_lines USING btree(length);")
#send_sql(conn, "CREATE INDEX landuse_boundary_lines_node_count_idx ON landuse_boundary_lines USING btree(node_count);")
#send_sql(conn, "CREATE INDEX landuse_boundary_lines_area_size_idx ON landuse_boundary_lines USING btree(area_size);")

Find polygons with long boundary but small area size:

In [ ]:
sql = '''SELECT
      area_id,
      feature,
      (sqrt(area_size)/length) AS size_length_ratio,
      area_size
    FROM landuse_boundary_lines
  ORDER BY size_length_ratio ASC NULLS LAST;
'''

Find polygons with many nodes per boundary length:

In [ ]:
sql = '''SELECT
        area_id,
        feature,
        area_size,
        (length/node_count::REAL) AS length_by_nodes,
        length,
        node_count
      FROM landuse_boundary_lines
      ORDER BY length_by_nodes ASC NULLS LAST;
'''

Unfortunately, the result is a bit worthless. The lowest ratio of length and node count is found at small circular areas (less than 1 m²). The largest ratio is found at rectangular forests (or rectangular grass areas on airports). This is not interesting.

In [ ]:
sql = '''SELECT
        area_id,
        feature,
        area_size,
        (node_count::REAL/area) AS nodes_per_area,
        length,
        node_count
      FROM landuse_boundary_lines
      ORDER BY length_by_nodes ASC NULLS LAST;
'''

The ratio of node count and area size does not return better results. The top and bottom entries of the list are almost identical.

Municipalities with many/few nodes per area:

In [ ]:
send_sql("CREATE INDEX IF NOT EXISTS landuse_boundary_lines_geom_idx ON landuse_boundary_lines USING gist(geom);")
sql = '''CREATE TABLE nodes_per_municipality AS
SELECT
    name,
    area_id,
    node_count/(ST_Area(area::geography) / 10000) AS nodes_per_area
  FROM (
    SELECT
        c.name,
        c.area_id,
        SUM(ST_NPoints(ST_Intersection(c.geom, b.geom))) AS node_count,
        c.geom AS area
      FROM cities AS c
      JOIN landuse_boundary_lines AS b
        ON c.geom && b.geom AND ST_Intersects(c.geom, b.geom)
      GROUP BY c.area_id, c.name, area
  ) AS a
  ORDER BY node_count DESC NULLS LAST;
'''
send_sql(conn, sql)

**TODO**

* determine node count per area and per boundary length, its distribution and differences between municipalities
* clip landuse boundaries to municipal polygons
* determine ratio of sqrt(area size) to boundary length, its distribution and differences between municipalities

## Road segments which are used by landuse polygons

Find landuse polygons sharing their boundary with roads.

In [ ]:
sql = '''
DROP TABLE IF EXISTS landuse_boundaries_on_roads;
SELECT
    ST_CollectionExtract(ST_Intersection(ST_Boundary(l.geom), r.geom), 2) AS geom,
    COALESCE(l.landuse, l."natural", l.water, l.waterway) AS land_feature,
    l.area_id AS land_id,
    COALESCE(r.highway, r.railway) AS road_feature,
    r.way_id AS road_id
  INTO landuse_boundaries_on_roads
  FROM land AS l
  JOIN streets AS r ON l.geom && r.geom AND ST_Intersects(l.geom, r.geom) AND ST_Relate(l.geom, r.geom, '***1*****');
'''
send_sql(conn, sql)

The table `landuse_boundaries_on_roads` contains duplicates if a road segment belongs to two landuse polygons. They are de-duplicated now:

In [ ]:
sql = '''
DROP TABLE IF EXISTS roads_as_landuse_boundaries;
CREATE INDEX IF NOT EXISTS landuse_boundaries_on_roads_road_id_feature ON landuse_boundaries_on_roads USING btree(road_id, road_feature);
SELECT
    ST_CollectionExtract(ST_Union(ST_ClusterIntersecting(geom)), 2) AS geom,
    road_feature,
    road_id
  INTO roads_as_landuse_boundaries
  FROM landuse_boundaries_on_roads
  GROUP BY road_id, road_feature;
'''
send_sql(conn, sql)

Get a geometry collection of the roads of each municipality.

Performance notes:

* In the `SELECT` part, `ST_Within` is faster than using `ST_CoveredBy` or `ST_Relate` (saves 25%).
* In the `JOIN` condition, not using `ST_Relate(rl.geom, c.geom, '1********')` saves additional 90% of time.

In [ ]:
send_sql(conn, "DROP TABLE IF EXISTS cities_road_network_length;")
sql = '''CREATE TABLE cities_road_network_length
  AS SELECT
    ST_Collect(ST_CollectionExtract(
      CASE
        WHEN ST_Within(rl.geom, c.geom) THEN rl.geom
        ELSE ST_Intersection(rl.geom, c.geom)
      END,
    2)) AS roads_geom,
    c.area_id AS area_id
  FROM streets AS rl
  JOIN cities AS c
    ON c.geom && rl.geom AND ST_Intersects(c.geom, rl.geom)
  GROUP BY c.area_id;
'''
send_sql(conn, sql)
send_sql(conn, "CREATE INDEX cities_road_network_length_area_id_idx ON cities_road_network_length USING btree(area_id);")

Intersect length of this road network per municpality and the total road network per municipality.

In [ ]:
send_sql(conn, "CREATE INDEX IF NOT EXISTS roads_as_landuse_boundaries_geom_idx ON roads_as_landuse_boundaries USING GIST(geom);")
send_sql(conn, "DROP TABLE IF EXISTS roads_as_landuse_boundaries_per_city;")
sql = """CREATE TABLE roads_as_landuse_boundaries_per_city AS
SELECT
      ST_Collect(ST_CollectionExtract(
        CASE
          WHEN ST_Within(rl.geom, c.geom) THEN rl.geom
          ELSE ST_Intersection(rl.geom, c.geom)
        END,
      2)) AS roads_geom,
      c.area_id AS area_id
    FROM cities AS c
    JOIN roads_as_landuse_boundaries AS rl
      ON
        c.geom && rl.geom
        AND ST_Intersects(c.geom, rl.geom)
    GROUP BY c.area_id;
"""
send_sql(conn, sql)

## A dataset of non-overlapping landuse

In OpenStreetMap, landuse polygons often overlap each other – often in the case when one landuse is a "hole" within another polygon (e.g. lake inside a forest). Before we can reliably determine a matrix of neighbouring landuse classes, we have to get rid of all overlaps.

In [ ]:
send_sql(conn, "DROP T